In [1]:
from docplex.cp.model import CpoModel, CpoStepFunction, INTERVAL_MIN, INTERVAL_MAX
import docplex.cp.utils_visu as visu
import os
import numpy as np
import pickle

In [2]:
from docplex.cp.config import context 
context.solver.local.execfile = "/Applications/CPLEX_Studio_Community129/cpoptimizer/bin/x86-64_osx/cpoptimizer"

In [4]:
dic = {}

i_list = np.arange(45,46)
j_list = np.arange(6,7)


for ii in i_list:
    for jj in j_list:
        
        filename = "./data/A30/j30rcp/J30" + str(ii) + '_' + str(jj) + ".RCP"
        with open(filename, "r") as file:
            NB_TASKS, NB_RESOURCES = [int(v) for v in file.readline().split()]
            CAPACITIES = [int(v) for v in file.readline().split()]
            TASKS = [[int(v) for v in file.readline().split()] for i in range(NB_TASKS)]

        # Extract duration of each task
        DURATIONS = [TASKS[t][0] for t in range(NB_TASKS)]

        # Extract demand of each task
        DEMANDS = [TASKS[t][1:NB_RESOURCES + 1] for t in range(NB_TASKS)]

        # Extract successors of each task
        SUCCESSORS = [TASKS[t][NB_RESOURCES + 2:] for t in range(NB_TASKS)]

        # Create model
        mdl = CpoModel()

        # Create task interval variables
        tasks = [mdl.interval_var(name="T{}".format(i + 1), size=DURATIONS[i]) for i in range(NB_TASKS)]

        # Add precedence constraints
        for t in range(NB_TASKS):
            for s in SUCCESSORS[t]:
                mdl.add(mdl.end_before_start(tasks[t], tasks[s - 1]))

        # Constrain capacity of resources
        for r in range(NB_RESOURCES):
            resources = [mdl.pulse(tasks[t], DEMANDS[t][r]) for t in range(NB_TASKS) if DEMANDS[t][r] > 0]
            mdl.add(mdl.sum(resources) <= CAPACITIES[r])

        # Minimize end of all tasks
        mdl.add(mdl.minimize(mdl.max([mdl.end_of(t) for t in tasks])))

        # Solve model
        msol = mdl.solve(TimeLimit=300)
        
        if msol.get_solve_status() == 'Optimal':
            print('{}\t{}\t{}\t{} sec'.format(ii, jj, msol.get_objective_values()[0], round(msol.get_solve_time(),4)))
            dic[ii,jj] = {}
            for k in range(1,33):
                dic[ii,jj][k] = msol.get_value('T' + str(k))
        
        else:
            print('{}\t{}\t****\t{} sec'.format(ii, jj, round(msol.get_solve_time(),4)))


45	6	129	103.5909 sec


In [ ]:
# with open('dic_fct.pickle', 'wb') as handle:
#     pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('dic_fct.pickle', 'rb') as handle:
    dic = pickle.load(handle)

In [9]:
msol.print_solution()

-------------------------------------------------------------------------------
Model constraints: 72, variables: integer: 0, interval: 32, sequence: 0
Solve status: Optimal, Fail status: SearchHasFailedNormally
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 103.59 sec
-------------------------------------------------------------------------------
Objective values: (129,), bounds: (129,), gaps: (0,)
T1: (start=0, end=0, size=0, length=0)
T2: (start=0, end=7, size=7, length=7)
T3: (start=18, end=27, size=9, length=9)
T4: (start=33, end=38, size=5, length=5)
T5: (start=7, end=12, size=5, length=5)
T6: (start=12, end=22, size=10, length=10)
T7: (start=44, end=51, size=7, length=7)
T8: (start=22, end=26, size=4, length=4)
T9: (start=28, end=33, size=5, length=5)
T10: (start=38, end=39, size=1, length=1)
T11: (start=12, end=18, size=6, length=6)
T12: (start=51, end=60, size=9, length=9)
T13: (start=27, end=37, size=10, length=10)
T14: (start=68, end=75, size

In [7]:
dic[45,6]

{1: (0, 0, 0),
 2: (0, 7, 7),
 3: (18, 27, 9),
 4: (33, 38, 5),
 5: (7, 12, 5),
 6: (12, 22, 10),
 7: (44, 51, 7),
 8: (22, 26, 4),
 9: (28, 33, 5),
 10: (38, 39, 1),
 11: (12, 18, 6),
 12: (51, 60, 9),
 13: (27, 37, 10),
 14: (68, 75, 7),
 15: (39, 44, 5),
 16: (75, 84, 9),
 17: (26, 28, 2),
 18: (44, 53, 9),
 19: (101, 104, 3),
 20: (61, 68, 7),
 21: (84, 88, 4),
 22: (53, 61, 8),
 23: (62, 71, 9),
 24: (61, 62, 1),
 25: (75, 83, 8),
 26: (84, 88, 4),
 27: (88, 98, 10),
 28: (98, 101, 3),
 29: (101, 111, 10),
 30: (111, 120, 9),
 31: (120, 129, 9),
 32: (129, 129, 0)}

In [5]:
dic

{(45, 6): {1: (0, 0, 0),
  2: (0, 7, 7),
  3: (18, 27, 9),
  4: (33, 38, 5),
  5: (7, 12, 5),
  6: (12, 22, 10),
  7: (44, 51, 7),
  8: (22, 26, 4),
  9: (28, 33, 5),
  10: (38, 39, 1),
  11: (12, 18, 6),
  12: (51, 60, 9),
  13: (27, 37, 10),
  14: (68, 75, 7),
  15: (39, 44, 5),
  16: (75, 84, 9),
  17: (26, 28, 2),
  18: (44, 53, 9),
  19: (101, 104, 3),
  20: (61, 68, 7),
  21: (84, 88, 4),
  22: (53, 61, 8),
  23: (62, 71, 9),
  24: (61, 62, 1),
  25: (75, 83, 8),
  26: (84, 88, 4),
  27: (88, 98, 10),
  28: (98, 101, 3),
  29: (101, 111, 10),
  30: (111, 120, 9),
  31: (120, 129, 9),
  32: (129, 129, 0)}}